In [2]:
import os
import sys
sys.path.insert(0, os.path.realpath('../'))

In [4]:
import LadiskDAQ
import numpy as np

In [6]:
# Create input and output tasks:

task_in = LadiskDAQ.NITask("TestInputTask_new", sample_rate=25600)
task_in.add_channel(channel_name="V0", device_ind=1, channel_ind=0, sensitivity=None, sensitivity_units=None, units="V", scale=1., min_val=-10, max_val=10.0)
task_in.add_channel(channel_name="V1", device_ind=1, channel_ind=1, sensitivity=None, sensitivity_units=None, units="V", scale=1., min_val=-10, max_val=10.0)

task_out = LadiskDAQ.NITaskOutput("TestOutputTask_new", sample_rate=25600)
task_out.add_channel(channel_name="V0_out", device_ind=2, channel_ind=0, min_val=-10, max_val=10)
task_out.add_channel(channel_name="V2_out", device_ind=2, channel_ind=1, min_val=-10, max_val=10)

Task name: input_task
Connected devices:
	(0) - VirtualDevice
	(1) - VirtualDevice2
Channels: ['V0', 'V1']

In [ ]:
#create signal:
fs = 25600 # output sample rate
mp = 10

t = np.arange(fs * mp) / fs  
signal1 = np.sin(2*np.pi*800*t) 
signal2 = np.sin(2*np.pi*200*t) 

excitation_signal = np.array([signal1, signal2]).T # exitation signal that generation class will use

In [ ]:
# create instances:
gen = LadiskDAQ.national_instruments.NIGeneration(task_out, excitation_signal)
acq = LadiskDAQ.national_instruments.NIAcquisition(task_in, acquisition_name="NI")
vis = LadiskDAQ.Visualization(refresh_rate=100)

# configure live visualization:
vis.add_lines((0,0), "NI", [0]) # Time signal V0
vis.add_lines((1,0), "NI", [1]) # Time signal V1
vis.add_lines((2,0), "NI", [0], function="fft") # Fourier transform of V0
vis.add_lines((3,0), "NI", [1], function="fft") # Fourier transform of V1

vis.config_subplot((0, 0), t_span=0.05, ylim=(-10, 10))
vis.config_subplot((1, 0), t_span=0.05, ylim=(-10, 10))
vis.config_subplot((2, 0), t_span=5.0, ylim=(0, 10), xlim=(200, 1000), refresh_rate=3000)
vis.config_subplot((3, 0), t_span=5.0, ylim=(0, 10), xlim=(200, 1000), refresh_rate=3000)

In [11]:
# create core object and add acquisition sources:
ldaq = LadiskDAQ.Core(acquisitions=[acq], generations=[gen], visualization=vis)

# run acquisition:
ldaq.run(10.)

In [12]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement

{'arduino': {'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 4.994e+00, 4.996e+00,
         4.998e+00]),
  'channel_names': ['channel 1', 'channel 2'],
  'data': array([[ 512.,  576.],
         [ 512.,  639.],
         [ 512.,  700.],
         ...,
         [-512.,  384.],
         [-512.,  447.],
         [ 512.,  512.]]),
  'sample_rate': 500},
 'esp': {'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 4.994e+00, 4.996e+00,
         4.998e+00]),
  'channel_names': ['esp CH 1', 'esp CH 2'],
  'data': array([[512., 263.],
         [512., 210.],
         [512., 162.],
         ...,
         [512., 449.],
         [512., 383.],
         [512., 324.]]),
  'sample_rate': 500}}